<a href="https://colab.research.google.com/github/WllmLin/NeuralStyleTransfer/blob/main/Style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

class VGG(nn.Module):
  def __init__(self):
    super(VGG, self).__init__()
    self.chosen_features = ['0','5','10','19','28']
    self.model = models.vgg19(pretrained=True).features[:29]
  def forward(self, x):
    features = []
    for layer_num, layer in enumerate(self.model):
      x = layer(x)
      if str(layer_num) in self.chosen_features:
        features.append(x)
    return features


def load_image(image_name):
  image = Image.open(image_name)
  image = loader(image).unsqueeze(0)
  return image.to(dev, torch.float)

image_size = 900
dev = torch.device('cuda')
loader = transforms.Compose(
    [
      transforms.Resize((image_size, image_size)),
      transforms.ToTensor(),
    ]
)


original_img = load_image('content.jpg').requires_grad_(False)
style_img = load_image('style.jpg').requires_grad_(False)
gen = original_img.clone().requires_grad_(True)

model = VGG().to(dev).eval()
# Hyperparameters

total_steps = 3000
learning_rate = 0.01
alpha = 0.9
beta = 0.1
optimizer = optim.Adam([gen], lr=learning_rate)

for step in range(total_steps):
  generated_features = model(gen)
  original_img_features = model(original_img)
  style_features = model(style_img)
  style_loss = content_loss = 0

  for gen_feature, ori_feature, style_feature in zip(
      generated_features, original_img_features, style_features
  ):
    batch_size, channel, height, width = gen_feature.shape
    content_loss += torch.mean((gen_feature - ori_feature) ** 2)

    G = gen_feature.view(channel, height*width).mm(
        gen_feature.view(channel, height*width).t()
    )

    A = style_feature.view(channel, height*width).mm(
        style_feature.view(channel, height*width).t()
    )
    style_loss += torch.mean( (G - A) ** 2 )
  total_loss = alpha * content_loss + beta * style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if step%200 == 0:
    print(total_loss)
    save_image(gen, f"process{int(step/200)}.jpg" )

tensor(63268872., device='cuda:0', grad_fn=<AddBackward0>)
tensor(594793.5625, device='cuda:0', grad_fn=<AddBackward0>)
